In [43]:
import pandas as pd

In [44]:
def payer_category(x):
    payers = {
        'Gross Charge': 'gross',
        'Maximum Contracted Payment': 'max',
        'Minimum Contracted Payment': 'min',
        'Self Pay': 'cash',
    }
    
    try:
        return payers[x]
    except KeyError:
        return 'payer'

In [45]:
file = '411889892_samedaysurgerycenterllc_standardcharges.xlsx'
df = pd.read_excel(file, dtype={'Px Code': str, 'CPT(R)/HCPCS Code': str})

In [46]:
df.rename(columns={
    'Px Code': 'internal_code',
    'Procedure Description': 'description',
    'CPT(R)/HCPCS Code': 'code'
}, inplace=True)

In [47]:
# Skip first row
df = df[1:]

In [48]:
cols = df.columns.tolist()
df = pd.melt(df, id_vars=cols[:3], value_vars=cols[3:], var_name='payer_orig', value_name='rate')

In [49]:
df['payer_name'] = df['payer_orig']
df['payer_category'] = df['payer_orig'].apply(payer_category)
df['rate_desc'] = 'Subject to multiple procedure discounting'
df['code_orig'] = df['code']
df['code_prefix'] = 'hcpcs_cpt'

In [50]:
df['hospital_ein'] = '411889892'
df['hospital_ccn'] = '430093'
df['filename'] = file
df['file_last_updated'] = '2022-12-01'
df['url'] = 'http://www.samedaysurgerycenter.org/uploads/411889892_samedaysurgerycenterllc_standardcharges.xlsx'

In [51]:
df.to_csv('sameday_surgery.csv', index=False)

,internal_code,description,code,payer_orig,rate,payer_name,payer_category,rate_desc,code_orig,code_prefix,hospital_ein,hospital_ccn,filename,file_last_updated,url
0,3601002100,FINE NEEDLE ASPIRATION BX W/O IMG GDN 1ST LESION,10021,Gross Charge,2020,Gross Charge,gross,Subject to multiple procedure discounting,10021,hcpcs_cpt,411889892,430093,411889892_samedaysurgerycenterllc_standardchar...,2022-12-01,http://www.samedaysurgerycenter.org/uploads/41...
1,3601006000,INCISION & DRAINAGE ABSCESS SIMPLE/SINGLE,10060,Gross Charge,2720,Gross Charge,gross,Subject to multiple procedure discounting,10060,hcpcs_cpt,411889892,430093,411889892_samedaysurgerycenterllc_standardchar...,2022-12-01,http://www.samedaysurgerycenter.org/uploads/41...
2,3601006100,INCISION & DRAINAGE ABSCESS COMPLICATED/MULTIPLE,10061,Gross Charge,2720,Gross Charge,gross,Subject to multiple procedure discounting,10061,hcpcs_cpt,411889892,430093,411889892_samedaysurgerycenterllc_standardchar...,2022-12-01,http://www.samedaysurgerycenter.org/uploads/41...
3,3601012000,INCISION & REMOVAL FOREIGN BODY SUBQ TISS SIMPLE,10120,Gross Charge,3869.25,Gross Charge,gross,Subject to multiple procedure discounting,10120,hcpcs_cpt,411889892,430093,411889892_samedaysurgerycenterllc_standardchar...,2022-12-01,http://www.samedaysurgerycenter.org/uploads/41...
4,3601012100,INCISION & REMOVAL FOREIGN BODY SUBQ TISS COMPL,10121,Gross Charge,3869.25,Gross Charge,gross,Subject to multiple procedure discounting,10121,hcpcs_cpt,411889892,430093,411889892_samedaysurgerycenterllc_standardchar...,2022-12-01,http://www.samedaysurgerycenter.org/uploads/41...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12271,7620000100,SSO PER HOUR,G0378,Maximum Contracted Payment,6271.2312,Maximum Contracted Payment,max,Subject to multiple procedure discounting,G0378,hcpcs_cpt,411889892,430093,411889892_samedaysurgerycenterllc_standardchar...,2022-12-01,http://www.samedaysurgerycenter.org/uploads/41...
12272,3900901600,RBC LEUKO REDUCED,P9016,Maximum Contracted Payment,254.7,Maximum Contracted Payment,max,Subject to multiple procedure discounting,P9016,hcpcs_cpt,411889892,430093,411889892_samedaysurgerycenterllc_standardchar...,2022-12-01,http://www.samedaysurgerycenter.org/uploads/41...
12273,3900905100,RBC LEUKO REDUCED CMV NEG,P9051,Maximum Contracted Payment,263.7,Maximum Contracted Payment,max,Subject to multiple procedure discounting,P9051,hcpcs_cpt,411889892,430093,411889892_samedaysurgerycenterllc_standardchar...,2022-12-01,http://www.samedaysurgerycenter.org/uploads/41...
12274,1100000200,SDSC-PRIVATE ROOM,NaN,Maximum Contracted Payment,1350.0,Maximum Contracted Payment,max,Subject to multiple procedure discounting,NaN,hcpcs_cpt,411889892,430093,411889892_samedaysurgerycenterllc_standardchar...,2022-12-01,http://www.samedaysurgerycenter.org/uploads/41...
